In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("Generalized").getOrCreate()

23/12/21 11:53:46 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/21 11:53:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/21 11:53:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/21 11:53:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
carros = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/Carros.csv", header=True, inferSchema=True, sep=";")
print(carros.count())
carros.show(5)

32
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [4]:
# Usando RFormula para separar o dataframe em conjunto de variaveis dependente e independentes
from pyspark.ml.feature import RFormula
Rformula = RFormula(formula="HP ~ Consumo + Cilindros + Cilindradas", featuresCol="independent_features", labelCol="dependent_variable")
carrosrf = Rformula.fit(carros).transform(carros)
carrosrf.select("independent_features", "dependent_variable").show(5, truncate=False)

+--------------------+------------------+
|independent_features|dependent_variable|
+--------------------+------------------+
|[21.0,6.0,160.0]    |110.0             |
|[21.0,6.0,160.0]    |110.0             |
|[228.0,4.0,108.0]   |93.0              |
|[214.0,6.0,258.0]   |110.0             |
|[187.0,8.0,360.0]   |175.0             |
+--------------------+------------------+
only showing top 5 rows



In [5]:
# Separando dados em Treino e Teste
CarrosTreino, CarrosTeste = carrosrf.randomSplit([0.8, 0.2])
print(CarrosTreino.count())
print(CarrosTeste.count())

25
7


In [10]:
from pyspark.ml.regression import GeneralizedLinearRegression
geral = GeneralizedLinearRegression(family="gaussian", featuresCol="independent_features", 
                                    labelCol="dependent_variable", link="identity", maxIter=100, regParam=0.08)
modelo = geral.fit(CarrosTreino)

In [11]:
previsao = modelo.transform(CarrosTeste)
previsao.select("dependent_variable", "prediction").show()

+------------------+------------------+
|dependent_variable|        prediction|
+------------------+------------------+
|             245.0| 209.8240521209849|
|             123.0| 133.6966159195552|
|             175.0| 205.2482073056182|
|             123.0|132.24066529648397|
|             175.0|204.40130215170123|
|              62.0| 66.80922036562633|
|             113.0| 64.78673595312874|
+------------------+------------------+



In [12]:
# Avaliando performance do modelo e medindo o erro
from pyspark.ml.evaluation import RegressionEvaluator

avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="dependent_variable", metricName="rmse")
rmse = avaliar.evaluate(previsao)
print(rmse)

28.19369949990621
